# MCR DAS WS2024 Homework 1

### Specifics:

This notebook was produced on a machine which runs the latest stable version of Ubuntu 22.04 running python3.12.1.

As such, in order to provide the best possible experience, it is recommended to run the notebook in a virtual environment, rather than directly on the machine.

Furthermore, the following script can be run to install the necessary dependencies (Find the contents of `requirements.txt` as an appendix to this assignment):

```sh
pip install setuptools && \
    pip install numpy && \
    pip install pandas && \
    pip install tqdm && \
    pip install joblib && \
    pip install category_encoders && \
    pip install polars && \
    pip install -r requirements.txt --no-build-isolation
```

In case you wanted to use `uv` as a package manager, this slightly adjusted script can be used instead:

```sh
uv pip install setuptools && \
    uv pip install numpy && \
    upv pip install pandas && \
    uv pi install tqdm && \
    uv pip install joblib && \
    uv pip install category_encoders && \
    uv pip install polars && \
    uv pip install mrmr-selection && \
    uv pip install -r requirements.txt --no-build-isolation
```

### Disclaimer

I have my own pipelines for datascience-related tasks, so I adapted them to this assignment.
As a direct consequence, I have not used pandas or numpy in this notebook.

Instead, I have used [polars](https://pola.rs/), which is a fast, in-memory dataframe library that is more suitable for my needs.

## Project Scope

The aim is to estimate the doctors' fees for a given dataset.

## Task 1: Data Analysis

#### A) Import the data from the “doctors_fee.csv” file into a dataframe.

In [ ]:
# Step 0: Imports

import polars as pl
import polars_ols as pls
from polars import read_csv
import altair as alt
import mrmr
from typing import overload, Literal
import pandas as pd
import statsmodels.formula.api as smf
from sklearn.metrics import r2_score

In [218]:
# Step 1: Import and format the data correctly

df = read_csv(
    "doctors_fees-v2.csv",
    separator=";",  # the separator is a semicolon
    decimal_comma=True,  # a comma was used to separate decimals
    schema={
        "ID": pl.UInt32,
        "Age": pl.UInt8,
        "Sex": pl.Categorical,  # I believe sex should be treated as categorical
        "BMI": pl.Float32,
        "Children": pl.UInt8,
        "Smoker": pl.String,
        "Region": pl.Categorical,
        "Charges": pl.Float64,
    },
).with_columns(pl.col("Smoker") == "yes")  # Match the string "yes" to the boolean True

#### B) How many rows and how many columns does the data frame have?

In [219]:
df.shape

(1338, 8)

##### Answer:

The dataframe has 1338 rows and 8 columns.

#### C) Delete the column id from the dataframe.

In [220]:
ids = df.drop_in_place(
    "ID"
)  # Keeping a reference to the ID column, while dropping it from the dataframe

#### D) Change the column names to lower case.

In [221]:
df = df.rename(str.lower)  # Python builtins trick

#### E) Change the column names “sex” to “gender”.

In [222]:
df = df.rename({"sex": "gender"})

#### F) In which columns are there how many missing values?


In [223]:
df.null_count()

age,gender,bmi,children,smoker,region,charges
u32,u32,u32,u32,u32,u32,u32
0,0,0,0,1,0,0


##### Answer:

There is 1 missing value in the `smokers` column.

#### G) If there are missing values, delete the corresponding rows from the dataframe.


In [224]:
df = df.drop_nulls()
df.shape  # Double check to see if exactly 1 row was purged

(1337, 7)

#### H) Replace „female“ with 0, and „Male“ with 1.

In [225]:
df = df.with_columns(
    (pl.col("gender") == "male").cast(pl.UInt8)
)  # Male was to be encoded with 1 so I had to do a comparison

#### I) Have you found incorrect values (which ones)?

I checked a data summary for my dataframe

In [226]:
df.describe()

statistic,age,gender,bmi,children,smoker,region,charges
str,f64,f64,f64,f64,f64,str,f64
"""count""",1337.0,1337.0,1337.0,1337.0,1337.0,"""1337""",1337.0
"""null_count""",0.0,0.0,0.0,0.0,0.0,"""0""",0.0
"""mean""",39.198953,0.504862,33.086456,1.147345,0.204936,null,13246.999375
"""std""",14.052113,0.500163,88.718895,2.306586,null,null,12117.303298
"""min""",18.0,0.0,15.96,0.0,0.0,null,-7151.092
"""25%""",27.0,0.0,26.315001,0.0,null,null,4719.73655
"""50%""",39.0,1.0,30.4,1.0,null,null,9377.9047
"""75%""",51.0,1.0,34.700001,2.0,null,null,16586.49771
"""max""",64.0,1.0,3267.0,73.0,1.0,null,63770.42801


And I found out that there was at least one clear outlier in the `bmi` column: the max was 3267.0, way out of proportion.

Furthermore, there's one charge of "-7151.092", which is an impossible value, and someone apparently has 73 children (which, while it might be possible, is so out of the ordinary that I'm going to assume it's a typo and thus exclude it).

By further analysing the region column, I gathered there's exactly one person from Germany.

In [227]:
germans = df.filter(pl.col("region") == "germany")

germans.shape

(1, 7)

For the sake of getting an actual data analysis through, I will perform task R right now, as to work with the data properly.

#### R) Delete faulty data records

In [228]:
df = df.filter(
    pl.col("region") != "germany",
    pl.col("bmi") < 55,  # Max BMI
    pl.col("children") < 7,  # Max number of children
    pl.col("charges") > 0,  # Max charges
)

df.shape

(1333, 7)

#### J) Estimate the distribution of age with a visualization. Comment on this distribution.

In [229]:
df.get_column("age").plot.hist(color="age:Q")

alt.Chart(...)

#### K) What is the distribution of gender? What is the distribution on smoker? What is the distribution on region? Comment these distributions.

In [230]:
gender_hist = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("gender:N", axis=alt.Axis(title="gender"), bin=alt.Bin(maxbins=2)),
        y=alt.Y("count()", axis=alt.Axis(title="count")),
        color="gender:N",
    )
    .properties(width=250)
)

gender_hist

alt.Chart(...)

There's an almost equal amount of males and females, so the data is balanced in that regard.

In [231]:
smokers_hist = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("smoker:N", axis=alt.Axis(title="smokers"), bin=alt.Bin(maxbins=2)),
        y=alt.Y("count()", axis=alt.Axis(title="count")),
        color="smoker:N",
    )
    .properties(width=250)
)

smokers_hist

alt.Chart(...)

The data presents around a 20% of smokers, which is quite a high percentage, but surpringly in line with Austrian average ([a quick research](https://globalactiontoendsmoking.org/research/tobacco-around-the-world/austria/) yielded around 22%).

In [232]:
regions_hist = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X("region:N", axis=alt.Axis(title="region")),
        y=alt.Y("count()", axis=alt.Axis(title="count")),
        color="region:N",
    )
    .properties(width=250)
)

regions_hist

alt.Chart(...)

Population is fairly well distributed, with a slight bias towards the southeast, which is generally in line with Austrian's population distribution.

#### L) Estimate the distribution of “bmi” with a visualization. Comment on this distribution

In [233]:
bmi_hist = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X(
            "bmi",
            axis=alt.Axis(title="bmi"),
            bin=alt.Bin(extent=(16, 55), step=5, maxbins=8),
        ),
        y=alt.Y("count()", axis=alt.Axis(title="count")),
    )
    .properties(width=250)
)

bmi_hist

alt.Chart(...)

The BMI distribution follows a normal distribution, with a slight left skew.

#### M) What is the distribution of the charges?

In [234]:
charges_hist = (
    alt.Chart(df)
    .mark_bar()
    .encode(
        x=alt.X(
            "charges",
            axis=alt.Axis(title="charges"),
            bin=alt.Bin(maxbins=5),
        ),
        y=alt.Y("count()", axis=alt.Axis(title="count")),
    )
    .properties(width=250)
)

charges_hist

alt.Chart(...)

More than 75% of charges were between 0 and 20,000, following an inverse-exponential distribution.

#### N) Create a correlation matrix (with the numeric variables including gender). Comment all correlation values. E.g.: on average, do women or mean have higher charges? Which correlations look strange?

In [235]:
df.select(pl.selectors.numeric()).corr()

age,gender,bmi,children,charges
f64,f64,f64,f64,f64
1.0,-0.021799,0.11013,0.042342,0.299469
-0.021799,1.0,0.046539,0.019684,0.057085
0.11013,0.046539,1.0,0.014025,0.199694
0.042342,0.019684,0.014025,1.0,0.067157
0.299469,0.057085,0.199694,0.067157,1.0


One interesting (positive) correlation is between age and charges. With a correlation of 0,3, this is not an extremely strong correlation, but it's still interesting to note.

Gender doesn't seem to be any indicator for charges (0.05), bmi is a weak indicator (0.20) but still relevant, while the number of childer is neglegible (0.06).

#### O) Create a scatterplot with „charges“ vs age. Comment this plot.


In [236]:
df.plot.scatter(x="age", y="charges")

alt.Chart(...)

We can see three main trends emerge from this scatterplot, with the evidence that an older age implies a higher charge.

This may also signify that similar pathologies may be more expensive in older age groups.

#### P) Create a „side-by-side-boxplot“ of your choice. Comment this plot.

In [237]:
chart = (
    alt.Chart(df)
    .transform_fold(["age", "bmi"], as_=["key", "value"])
    .mark_boxplot()
    .encode(x="key:N", y="value:Q")
    .properties(width=250)
)


chart

alt.Chart(...)

I decided to compare the age and bmi of the population.

We can see that most of the bmi interquartile range is between 26 and 34, with many outliers coming above 50.
Age is, instead, well distributed, with a median of 39 and no determined outliers.

#### Q) Create a new column „bmi_age“ – as the multiplication of bmi and age.

In [238]:
df = df.with_columns(
    (pl.col("bmi") * pl.col("age")).alias("bmi_age"),
)

df.head()

age,gender,bmi,children,smoker,region,charges,bmi_age
u8,u8,f32,u8,bool,cat,f64,f32
19,0,27.9,0,true,"""southwest""",16884.924,530.099976
18,1,33.77,1,false,"""southeast""",1725.5523,607.859985
28,1,33.0,3,false,"""southeast""",4449.462,924.0
33,1,22.705,0,false,"""northwest""",21984.47061,749.265015
32,1,28.879999,0,false,"""northwest""",3866.8552,924.159973


## Task 2: Linear Regression

Before we define any tasks, I'll import my usual data analysis routine for train-test split and model evaluation.

In [239]:
@overload
def mrmr_regression(
    df: pl.DataFrame, target_column: str, k: int, return_scores: Literal[True]
) -> tuple[list[str], pd.Series, pd.DataFrame]: ...


@overload
def mrmr_regression(
    df: pl.DataFrame, target_column: str, k: int, return_scores: Literal[False]
) -> list[str]: ...


type MRMR = list[str] | tuple[list[str], pd.Series, pd.DataFrame]


def mrmr_regression(
    df: pl.DataFrame, target_column: str, k: int, return_scores: bool
) -> MRMR:
    return mrmr.polars.mrmr_regression(
        df=df, target_column=target_column, K=k, return_scores=return_scores
    )


def f_selection(df: pl.DataFrame, target_column: str, features: list[str]) -> pd.Series:
    return mrmr.polars.f_regression(
        df=df, target_column=target_column, features=features
    )


def train_test_split_df(df: pl.DataFrame, seed: int = 0, test_size: float = 0.25):
    return df.with_columns(
        pl.int_range(pl.len(), dtype=pl.UInt32)
        .shuffle(seed=seed)
        .gt(pl.len() * test_size)
        .alias("split")
    ).partition_by("split", include_key=False)


def train_test_split(
    X: pl.DataFrame, y: pl.DataFrame, seed: int = 0, test_size: float = 0.2
):
    X_train, X_test = train_test_split_df(X, seed=seed, test_size=test_size)
    y_train, y_test = train_test_split_df(y, seed=seed, test_size=test_size)
    return X_train, X_test, y_train, y_test

#### A) Split the dataframe from Task 1 into a training part and a test part in the ratio 75% to 25%.

In [ ]:
train, test = train_test_split_df(
    df.with_columns(
        pl.col("gender").cast(pl.UInt8),
        pl.col("smoker").cast(pl.UInt8),
        pl.col("region").rank("dense").alias("region"),
    ),
)

In [242]:
lm_01 = smf.ols(formula="charges ~ bmi", data=train).fit()
lm_02 = smf.ols(formula="charges ~ bmi + gender", data=train).fit()
lm_03 = smf.ols(formula="charges ~ bmi + smoker", data=train).fit()
lm_04 = smf.ols(
    formula="charges ~ age + gender + bmi + children + smoker + region + bmi_age",
    data=train,
).fit()

#### C) For lm02:<br>What are values for R2? Comment these values.<br>Which features are significant?<br>Write the regression equation as a formula (rounding is allowed)

In [243]:
lm_02.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.054
Method:                 Least Squares   F-statistic:                     10.55
Date:                Wed, 04 Dec 2024   Prob (F-statistic):           3.63e-05
Time:                        07:24:35   Log-Likelihood:                -3585.0
No. Observations:                 334   AIC:                             7176.
Df Residuals:                     331   BIC:                             7187.
Df Model:                           2                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1846.4205   3211.110     -0.575      0.566   -8163.177    4470.336
bmi          462.2886    101.138      4.571      0.000     263.334     661.243
gender       167.6320   1223.448      0.137      0.891   -2239.082    2574.346
==============================================================================
Omnibus:                       78.792   Durbin-Watson:                   1.942
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              132.936
Skew:                           1.377   Prob(JB):                     1.36e-29
Kurtosis:                       4.403   Cond. No.                         167.
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

##### Model Overview
- Dependent Variable: charges
- R-squared: 0.060 (6% of variance explained by the model)
- Adjusted R-squared: 0.054
- F-statistic: 10.55 (p-value: 3.63e-05, indicating overall model significance)

##### Coefficients
1. Intercept: -1846.4205 (p-value: 0.566, not significant)
2. BMI: 462.2886 (p-value: 0.000, highly significant)
3. Gender: 167.6320 (p-value: 0.891, not significant)

##### Significant Features
Based on the p-values, only BMI is statistically significant (p < 0.05).

##### Regression Equation
`charges = -1846.4205 + 462.2886 * BMI + 167.6320 * gender`

Note: Although gender is included in the equation, it's not statistically significant.

##### Model Fit
The low R-squared value (0.060) suggests that the model explains only 6% of the variance in charges, indicating a poor fit.

#### D) For lm04:<br>What are values for R2? Comment these values.<br>Which features are significant?<br>Write the regression equation as a formula (rounding is allowed).<br>How big is the expected increase in charges if a person has one more child?

In [244]:
lm_04.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:                charges   R-squared:                       0.709
Model:                            OLS   Adj. R-squared:                  0.703
Method:                 Least Squares   F-statistic:                     113.5
Date:                Wed, 04 Dec 2024   Prob (F-statistic):           1.84e-83
Time:                        07:24:35   Log-Likelihood:                -3389.2
No. Observations:                 334   AIC:                             6794.
Df Residuals:                     326   BIC:                             6825.
Df Model:                           7                                         
Covariance Type:            nonrobust                                         
==============================================================================
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept  -1.042e+04   5556.399     -1.876      0.062   -2.14e+04     506.295
age          192.8789    138.340      1.394      0.164     -79.272     465.030
gender     -1346.3137    689.196     -1.953      0.052   -2702.147       9.520
bmi          351.1991    181.636      1.934      0.054      -6.128     708.527
children     556.8695    296.922      1.875      0.062     -27.255    1140.994
smoker      2.334e+04    895.209     26.069      0.000    2.16e+04    2.51e+04
region      -362.0221    315.723     -1.147      0.252    -983.134     259.090
bmi_age        1.0954      4.358      0.251      0.802      -7.478       9.669
==============================================================================
Omnibus:                      101.668   Durbin-Watson:                   2.069
Prob(Omnibus):                  0.000   Jarque-Bera (JB):              251.120
Skew:                           1.473   Prob(JB):                     2.95e-55
Kurtosis:                       6.061   Cond. No.                     2.15e+04
==============================================================================

Notes:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 2.15e+04. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

##### Model Overview
- Dependent Variable: charges
- R-squared: 0.711 (71.1% of variance explained by the model)
- F-statistic: 114.7 (p-value: 5.40e-84, indicating high overall model significance)

##### R-squared Values
- R-squared: 0.711
- Adjusted R-squared: 0.705

These values indicate that the model explains about 71% of the variance in charges, which is a good fit. The adjusted R-squared is close to the R-squared, suggesting that the additional predictors are contributing meaningfully to the model.

##### Significant Features (p < 0.05)
1. Intercept (p = 0.018)
2. BMI (p = 0.035)
3. Smoker (p = 0.000)

##### Borderline Significant Features (0.05 < p < 0.10)
1. Gender (p = 0.057)
2. Children (p = 0.062)
3. Region (p = 0.052)

##### Non-significant Features
1. Age (p = 0.123)
2. BMI_Age interaction (p = 0.956)

##### Regression Equation
`charges = -13730 + 214.0058 * age - 1313.6689 * gender + 386.5574 * bmi + 552.6163 * children + 23200 * smoker + 635.9918 * region + 0.2391 * bmi_age`

##### Expected Increase in Charges for One Additional Child
The coefficient for 'children' is 552.6163, meaning that for each additional child, the expected increase in charges is $552.62, holding all other variables constant. However, this feature is borderline significant (p = 0.062).

#### e) Read the second note in the summary table of lm04.<br>What does that mean and what can we do?

The second note says:

`[2] The condition number is large, 2.24e+04. This might indicate that there are strong multicollinearity or other numerical problems.`

Which means there are 2 or more features which are strongly correlated with each other. We could remove one of the correlated features to reduce multicollinearity.

#### F) Create the fitted vs target plot for lm02. Comment this plot.

In [245]:
transformed = train.with_columns(
    pl.col("charges").least_squares.ols(pl.col("bmi")).alias("regression")
).select("bmi", "charges", "regression")

chart = (
    alt.Chart(transformed)
    .mark_point()
    .encode(x=alt.X("bmi", scale=alt.Scale(domain=(19, 50))), y="charges")
)

chart + alt.Chart(transformed).mark_line(color="crimson").encode(
    x="bmi", y="regression"
)

alt.LayerChart(...)

The linear relation is not very clear, but there is a linear trend. The model is underfitting the data.

#### G) Create the fitted vs target plot for lm04. Comment this plot.

In [246]:
transformed = train.with_columns(
    pl.col("charges")
    .least_squares.ols(
        pl.col("bmi"),
        pl.col("gender"),
        pl.col("smoker"),
        pl.col("region"),
        pl.col("bmi_age"),
        pl.col("age"),
        pl.col("children"),
    )
    .alias("regression")
)

transformed.plot.scatter(x="regression", y="charges")

alt.Chart(...)

Here a linear relation is much clearer, the model can be easily described by tracing a relative straight line.

#### H) lm04: comment the values of R2 with the training data and the test data, and their relationship.

In [247]:
predict = lm_04.predict(test)

t = test.get_column("charges")

type(predict)

r2_score(t, predict), r2_score(train.get_column("charges"), lm_04.predict(train))

(0.7599997070173443, 0.7089987208327742)

By comparing the test r squared score with the training r squared score, we can see that the model is explains 7% of the variance in the training data and 7.5% of the variance in the test data.

This means that the model is a good fit to serve as a predictor for price oscillation.

#### I) Lm04: plot the residual plot. Comment the plot.

In [248]:
dfd = pl.DataFrame().with_columns(
    t.alias("predictions"),
    (pl.from_pandas(predict) - t).alias("residual"),
)

scatter = alt.Chart(dfd).mark_circle().encode(x="predictions", y="residual")

scatter

alt.Chart(...)

The plot reveals significant problems with the model's assumptions:

Non-constant Variance: The spread of the residuals is not consistent across the range of predictions. There's much greater variability in residuals at higher prediction values.

Non-linearity: The pattern of residuals isn't randomly scattered around zero. Instead, there are distinct clusters and a clear non-random pattern.

Potential Outliers: There are some points that are far away from the main cluster of residuals.

#### J) Predict the charges of one person with the following code:

```
data = {'age': [10], 'sex':[0], 'bmi': [32], 'children': [0], 'smoker': ['no'],
'region': ['southeast'], 'bmi_age': [320]}
```

In [249]:
data = {
    "age": [10],
    "gender": [0],
    "bmi": [32],
    "children": [0],
    "smoker": [0],  # encoded the smoker status
    "region": [1],  # encoded the region
    "bmi_age": [320],
}

df1 = pl.DataFrame(data)

lm_04.predict(df1)

0    2731.043276
dtype: float64

The increase is calculated as almost 1500 euros (1496.55).

## Appendix A: Requirements

Paste the following into a file called `requirements.txt` to ensure you have all the right dependencies:

```r
altair==5.5.0
asttokens==2.4.1
attrs==24.2.0
category-encoders==2.6.4
comm==0.2.2
debugpy==1.8.9
decorator==5.1.1
executing==2.1.0
ipykernel==6.29.5
ipython==8.29.0
jedi==0.19.2
jinja2==3.1.4
joblib==1.4.2
jsonschema==4.23.0
jsonschema-specifications==2024.10.1
jupyter-client==8.6.3
jupyter-core==5.7.2
markupsafe==3.0.2
matplotlib-inline==0.1.7
mrmr-selection==0.2.8
narwhals==1.14.2
nest-asyncio==1.6.0
numpy==2.1.3
packaging==24.2
pandas==2.2.3
parso==0.8.4
patsy==1.0.1
pexpect==4.9.0
platformdirs==4.3.6
polars==1.15.0
prompt-toolkit==3.0.48
psutil==6.1.0
ptyprocess==0.7.0
pure-eval==0.2.3
pygments==2.18.0
python-dateutil==2.9.0.post0
pytz==2024.2
pyzmq==26.2.0
referencing==0.35.1
rpds-py==0.21.0
scikit-learn==1.5.2
scipy==1.14.1
setuptools==75.6.0
six==1.16.0
stack-data==0.6.3
statsmodels==0.14.4
threadpoolctl==3.5.0
tornado==6.4.2
tqdm==4.67.1
traitlets==5.14.3
typing-extensions==4.12.2
tzdata==2024.2
wcwidth==0.2.13
```
